In [1014]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from data_explorer.visualisation import scatter_figure_byTarget, make_heatmap, view_unique_values, hist_distributions, make_cat_heatmap, view_values_frequency, make_mix_pipeline, calc_outliers
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn import set_config

# Import des packages, Chargement du dataset, fonction de vision de base

In [1015]:
df = pd.read_csv('wines_SPA.csv')
df.drop_duplicates(inplace=True)

In [1016]:
df.shape

(2048, 11)

Nous avons un data set de 2048 lignes et 11 colonnes
Observons les infos:

In [1017]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2048 entries, 0 to 6100
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   winery       2048 non-null   object 
 1   wine         2048 non-null   object 
 2   year         2046 non-null   object 
 3   rating       2048 non-null   float64
 4   num_reviews  2048 non-null   int64  
 5   country      2048 non-null   object 
 6   region       2048 non-null   object 
 7   price        2048 non-null   float64
 8   type         1942 non-null   object 
 9   body         1777 non-null   float64
 10  acidity      1777 non-null   float64
dtypes: float64(4), int64(1), object(6)
memory usage: 192.0+ KB


In [1018]:
df.isnull().sum()

winery           0
wine             0
year             2
rating           0
num_reviews      0
country          0
region           0
price            0
type           106
body           271
acidity        271
dtype: int64

2 valeurs nulles dans 'year' qui sont nulles

Après avoir vérifié les prix pratiqué en fonction des années, les vins portant le même nom et le nombre de review, j'en ai conclu que ces deux éléments ne sont pas remplaçable par une valeur. (une valeur mediane ou moyenne serait abérrante puisque l'un des vins est tout seul et l'autre se situe sur un intervalle trop grand entre de 1980 à 2021, or la qualité dépend de l'année du cépage)

Les valeurs N.V. n'étaient pas remplaçable ou conduiraient à des duplicatas

In [1019]:
df = df.dropna(subset=['year'])
df = df.drop(df[df['year'] == 'N.V.'].index)
df['year'] = df['year'].astype(float)

In [1020]:
ret = {}
for col in df.columns:
    ret[col] = df[col].unique()


In [1021]:
print(df.loc[calc_outliers(df.price, 0.05,0.95).index].to_string())
print()
print(df.loc[calc_outliers(df.year, 0.20,0.80).index].to_string())
print()
print(df.loc[calc_outliers(df.num_reviews, 0.20,0.80).index].to_string())

                           winery                            wine    year  rating  num_reviews country            region    price                  type  body  acidity
16                   Vega Sicilia                           Unico  1962.0     4.8          295  Espana  Ribera del Duero  1620.00  Ribera Del Duero Red   5.0      3.0
83              Dominio de Pingus                          Pingus  2004.0     4.7          285  Espana  Ribera del Duero  1500.00  Ribera Del Duero Red   5.0      3.0
92                   Vega Sicilia                           Unico  1968.0     4.7          225  Espana  Ribera del Duero  2087.25  Ribera Del Duero Red   5.0      3.0
97                   Vega Sicilia  Unico Reserva Especial Edicion  1995.0     4.7          196  Espana  Ribera del Duero  1500.00  Ribera Del Duero Red   5.0      3.0
98              Dominio de Pingus                          Pingus  1999.0     4.7          184  Espana  Ribera del Duero  1715.18  Ribera Del Duero Red   5.0      3.

J'élimine les outliers importants (bouteille très très chère, très anciennes)

In [1022]:
df = df.drop(calc_outliers(df.price, 0.05,0.95).index)
df = df.drop(calc_outliers(df.year, 0.20,0.80).index)
# df = df.drop(calc_outliers(df.num_reviews, 0.10,0.90).index)
df = df.drop(['country', 'acidity'],axis=1)

In [1023]:
# for col in df.columns:
#     df[col].hist()
#     plt.title(col)
#     plt.show()
    

# Observation des distributions en fonctions des colonnes, choix de la target et des éventuelles features

In [1024]:
heatmap = df.reset_index(drop=True)
cat = df.select_dtypes(object).columns.to_list()
for i in cat:
    heatmap[i] = df[i].factorize()[0]
heatmap

,winery,wine,year,rating,num_reviews,region,price,type,body
0,0,0,2013.0,4.9,58,0,995.00,0,5.0
1,1,1,2018.0,4.9,31,1,313.50,1,4.0
2,2,2,2009.0,4.8,1793,2,324.95,2,5.0
3,2,2,1999.0,4.8,1705,2,692.96,2,5.0
4,2,2,1998.0,4.8,1209,2,490.00,2,5.0
...,...,...,...,...,...,...,...,...,...
1820,187,459,2017.0,4.2,390,43,24.45,3,4.0
1821,42,42,2011.0,4.2,389,2,64.50,2,5.0
1822,123,739,2016.0,4.2,388,2,31.63,2,5.0
1823,12,436,2005.0,4.2,384,5,73.00,5,4.0


In [1025]:
# sns.heatmap(heatmap.corr(), annot=True, fmt='.2f')

In [1026]:
# make_cat_heatmap(df,['winery','wine','region','type'])

# Vérifications du nombres de valeurs uniques


In [1027]:
# Vérifications du nombres de valeurs uniques
view_unique_values(df, count=True)

column 0 : winery => 438 uniques values
column 1 : wine => 761 uniques values
column 2 : year => 25 uniques values
column 3 : rating => 8 uniques values
column 4 : num_reviews => 785 uniques values
column 5 : region => 71 uniques values
column 6 : price => 1160 uniques values
column 7 : type => 21 uniques values
column 8 : body => 5 uniques values


### Au vue des premières analyses, je décide de choisir en target (y) le 'rating' pour effectuer ma prédiction
### Au vue des distributions, je choisis comme première feature (X), la colonne "winery

## 1ère itération : Linear Regression
- target = "rating"
- feature = "winery"

In [1028]:
y = df[['price']]
X = df[['winery', 'wine']]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

first_pred = make_mix_pipeline()
first_pred.fit(X_train, y_train)
first_score = first_pred.score(X_test, y_test)
first_cv = cross_validate(first_pred, X_train, y_train, cv=5)['test_score'].mean()
print(f"score de prédiction sur le jeu de test :{first_score}")
print(f"score de cross-validation : {first_cv}")

score de prédiction sur le jeu de test :0.8202528324800296
score de cross-validation : 0.7627490945172122


### Résultat encoureageant, je fais tester d'ajouter des features pour la 2ème itérations 

## 2ème itération : Linear Regression
- target = "rating"
- feature = "winery", "wine"

In [1029]:
y_2 = df[['price']]
X_2 = df[['winery','wine']]

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2,y_2, test_size=0.3, random_state=42)

second_pred = make_mix_pipeline()
second_pred.fit(X_train_2, y_train_2)
second_score = second_pred.score(X_test_2, y_test_2)
second_cv = cross_validate(second_pred, X_train_2, y_train_2, cv=5)['test_score'].mean()
print(f"score de prédiction sur le jeu de test :{second_score}")
print(f"score de cross-validation : {second_cv}")

score de prédiction sur le jeu de test :0.8202528324800296
score de cross-validation : 0.7627490945172122


Il y a une amélioration du modèle, le score cross-validation est proche du score de prédiction, notre modèle est donc plutôt généralisable.  
Je vais de cleaner les outliers sur mes features afin de voir si cela améliore le modèle.  

In [1030]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1825 entries, 0 to 6100
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   winery       1825 non-null   object 
 1   wine         1825 non-null   object 
 2   year         1825 non-null   float64
 3   rating       1825 non-null   float64
 4   num_reviews  1825 non-null   int64  
 5   region       1825 non-null   object 
 6   price        1825 non-null   float64
 7   type         1728 non-null   object 
 8   body         1570 non-null   float64
dtypes: float64(4), int64(1), object(4)
memory usage: 142.6+ KB


In [1031]:
# Je regarde la distributions des valeurs uniques sur ma features winery
# Je m'aperçois que sur 425 valeurs possibles, les 105 valeurs qui apparaissent le plus, 
# c'est à dire au minimun 4 fois, représente 91% de la distributions
# environ 25% des valeurs uniques de 'winery' => apparaissent 4 fois minimum => represente 91 % de la distribution. 
print(view_values_frequency(df.wine).head(116).sum())
print(len(df.wine.value_counts()))


most_wine_appear = df.wine.value_counts().head(90).index.tolist()
df = df.loc[df['wine'].isin(most_wine_appear)]
df
print(df.shape[0]/7500)

48.05479452054796
761
0.10306666666666667


In [1032]:
# Je regarde la distributions des valeurs uniques sur ma features 'wine'
# Je m'aperçois que sur 335 valeurs possibles, les 1 valeurs qui apparaissent le plus
# c'est à dire au minimun 4 fois, représente 91% de la distributions
# environ 34% des valeurs uniques de 'wine' => apparaissent 4 fois minimum => represente 93 % de la distribution.
# Je sais egalement, qu'après avoir supprimé les valeurs nulles et ces 2 series d'outliers, il ne me restera que 0,72% des données de mon dataset de base
print(view_values_frequency(df.wine).head(116).sum())
print(len(df.wine.value_counts()))


most_wine_appear = df.wine.value_counts().head(90).index.tolist()
df = df.loc[df['wine'].isin(most_wine_appear)]
df
print(df.shape[0]/7500)

100.0
90
0.10306666666666667


## 3ème itération : Linear Regression
- target = "price"
- feature = 'winery','wine','year','rating'

In [1033]:

y_3 = df[['price']]
X_3 = df[['winery','wine','year','rating']]

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_3,y_3, test_size=0.3, random_state=42)


third_pred = make_mix_pipeline()
third_pred.fit(X_train_3, y_train_3)
third_score = third_pred.score(X_test_3, y_test_3)
third_cv = cross_validate(third_pred, X_train_3, y_train_3, cv=5)['test_score'].mean()
print(f"score de prédiction sur le jeu de test : {third_score}")
print(f"score de cross-validation : {third_cv}")

score de prédiction sur le jeu de test : 0.9104178375363485
score de cross-validation : 0.8807207688106373


Après plusieurs tests supplémentaires (ajout de features), je n'obtiens pas de meilleur résultats.  
On pourrait eventuellement tester d'autre technique d'encodage ou améliorer le cleaning du dataset

## 4ème itération : Ridge Regression
- target = "rating"
- feature = "winery", "wine"

---
# Target: Rating

La deuxième target qui nous intéresse, la note attribuée au vin.  
Après quelques tests, le scoring semble être cohérent avec les même features que pour le prix  
Après quelques test nous décidons de conserver les premières features de bases puis dans rajouter quelques unes  

---
# Target: Rating

La deuxième target qui nous intéresse, la note attribuée au vin.  
Après quelques tests, le scoring semble être cohérent avec les même features que pour le prix  
Après quelques test nous décidons de conserver les premières features de bases puis dans rajouter quelques unes  

body et num_review améliorent un tout petit peu le modèle
Je vais donc clean en amont le dataset pour l'améliorer

body et num_review améliorent un tout petit peu le modèle
Je vais donc clean en amont le dataset pour l'améliorer

In [1034]:
y_4 = df[['rating']]
X_4 = df[['winery','wine','year','price', 'num_reviews']]


X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X_4,y_4, test_size=0.3, random_state=42)


third_pred = make_mix_pipeline(model=Ridge(alpha=0.4))
third_pred.fit(X_train_4, y_train_4)
third_score = third_pred.score(X_test_4, y_test_4)
third_cv = cross_validate(third_pred, X_train_4, y_train_4, cv=5)['test_score'].mean()
print(f"score de prédiction sur le jeu de test : {third_score}")
print(f"score de cross-validation : {third_cv}")

score de prédiction sur le jeu de test : 0.6889816117598011
score de cross-validation : 0.5951789712079482
